# Обработка ОКС

Описать методику выполнения

## 1. Чтение файлов

In [302]:
# Расширяем режим просмотра Jupyter
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 90% !important; }</style>"))

In [303]:
import time
start_time = time.time()

In [304]:
# Стандартные библиотеки
import math
import os
import re

# Сторонние библиотеки
import pandas as pd
from tqdm import tqdm
import xlwings as xw

In [305]:
# # Путь к файлам на маке
# DIR = '/Users/romanbespalov/ЦКО/Паша/Исходные данные/'

# # Путь пересохраненных файлов
# RESAVED_DIR = os.path.join(DIR, 'resaved/')

# Путь к файлам на рабочем компьютере
DIR = 'L:/Беспалов Р.А/Паша/'

# Путь пересохраненных файлов
RESAVED_DIR = os.path.join(DIR, 'resaved/')

# Путь готовых файлов
MODIFIED_DIR = os.path.join(DIR, 'modified/')

# Установим рабочий каталог
os.chdir(DIR)

In [306]:
# Получаем список файлов
FILES = os.listdir(DIR)

def resave_excel(files):
    """Пересохраняет ексель файлы."""
    for file in tqdm(files):
        if file.rsplit('.', 1)[-1] == 'xlsx':
            wb = xw.Book(os.path.join(DIR, file))
            wb.save(os.path.join(RESAVED_DIR, file))
            wb.close()

# resave_excel(FILES)

In [307]:
# Получаем список пересохраненный файлов
RESAVED_FILES = os.listdir(RESAVED_DIR)

def excel_to_df(files):
    """Создает датафреймы из ексель файлов."""
    data = []
    for file in files:
        if file.rsplit('.', 1)[-1] == 'xlsx':
            df = pd.read_excel(os.path.join(RESAVED_DIR, file))
            df.at[3, 'Имя файла'] = 'Имя файла'
            df.at[5, 'Имя файла'] = file
            data.append(df)
    return data

list_df = excel_to_df(RESAVED_FILES)

In [308]:
df_1 = list_df[0]

## 2. Форматирование

In [309]:
def prapare_data(df):
    """Подготовка датафрейма."""
    df.drop(df.index[[0, 1, 2, 4]], inplace=True)
    df = df.reset_index(drop=True)
    df.columns = df.iloc[0]
    df = df.iloc[1:]
    df = df.reset_index(drop=True)
    return df

df_1 = prapare_data(df_1)

In [310]:
set_list = set(['Из прочих материалов', 'Смешанные', 'Иное'])


def compare_walls(string_1, string_2):
    """Сравнивает 2 поля - Пред.: Материал наружных стен и Материал наружных стен.
    Делит строку по «;», убирает пробелы и создает множество.
    """
    try:
        # проверка что поля имеют разный тип данных
        if isinstance(string_1, float) and isinstance(string_2, str) or isinstance(string_2, float) and isinstance(string_1, str):
            if isinstance(string_1, float) and isinstance(string_2, str):
                values_2 = set(map(str.strip, string_2.split(';')))
                values_2.discard('')
                if set_list.intersection(values_2):
                    result = 'Вернуть материал стен, как в предыдущем перечне.'
                else:
                    result = None
            else:
                result = None
        # проверка что поля имеют одинаковый тип данных - nan(Not a Number) value
        elif isinstance(string_1, float) and isinstance(string_2, float):
            result = None
        # проверка что поля имеют одинаковый тип данных str
        elif type(string_1) == str and type(string_2) == str:
            # создание множества из строки
            values_1 = set(map(str.strip, string_1.split(';')))
            values_2 = set(map(str.strip, string_2.split(';')))
            # удаляем пустые элементы
            values_1.discard('')
            values_2.discard('')
            if values_1 != values_2 and set_list.intersection(values_2):
                result = 'Вернуть материал стен, как в предыдущем перечне.'
            else:
                result = None
    except Exception as e:
        print(e)
    return result

In [311]:
def step_0(df):
    """Создает дополнительное поле «Комментарий»."""
    old_column = 'Сравнение: Материал наружных стен'
    new_column = 'Комментарий'
    previous_column = 'Пред.: Материал наружных стен'
    present_column = 'Материал наружных стен'
    
    df.insert(92, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: compare_walls(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else None, axis=1)
    return df

df_1 = step_0(df_1)

In [312]:
def compare_class(string_1, string_2):
    """Сравнивает 2 поля - Пред.: Класс конструктивной системы и Класс конструктивной системы.
    Делит строку по «;», убирает пробелы и создает множество.
    """
    try:
        # проверка что поля имеют разный тип данных
        if isinstance(string_1, float) and isinstance(string_2, str) or isinstance(string_2, float) and isinstance(string_1, str):
            result = 'Различны'
        # проверка что поля имеют одинаковый тип данных - nan(Not a Number) value
        elif isinstance(string_1, float) and isinstance(string_2, float):
            result = 'Без изменений'
        # проверка что поля имеют одинаковый тип данных str
        elif type(string_1) == str and type(string_2) == str:
            # создание множества из строки
            values_1 = set(map(str.strip, string_1.split(';')))
            values_2 = set(map(str.strip, string_2.split(';')))
            # удаляем пустые элементы
            values_1.discard('')
            values_2.discard('')
            if values_1 == values_2:
                result = 'Без изменений'
            else:
                result = 'Различны'
    except Exception as e:
        print(e)
    return result

In [313]:
def value_replace(value):
    """Заменяет параметр в поле."""
    if value == 'Новое':
        value = 'Различны'
    elif value == 'Удалено':
        value = 'Без изменений'
    return value

In [314]:
def step_1(df):
    """Сравнение: Класс конструктивной системы."""
    old_column = 'Сравнение: Класс конструктивной системы'
    new_column = 'Сравнение: Класс конструктивной системы temp'
    previous_column = 'Пред.: Класс конструктивной системы'
    present_column = 'Класс конструктивной системы'

    df.insert(96, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: compare_class(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else value_replace(row[old_column]), axis=1)
    return df

def step_2(df):
    """Сравнение: Материал наружных стен."""
    new_column = 'Сравнение: Материал наружных стен temp'

    df.insert(92, new_column, pd.Series)
    df[new_column] = df['Сравнение: Класс конструктивной системы temp']
    return df

df_1 = step_1(df_1)
df_1 = step_2(df_1)

In [315]:
def step_3(df):
    """Сравнение: Основные характеристики (описание)."""
    old_column = 'Сравнение: Основные характеристики (описание)'
    new_column = 'Сравнение: Основные характеристики (описание) temp'
    previous_column = 'Пред.: Основные характеристики (описание)'
    present_column = 'Основные характеристики (описание)'

    df.insert(89, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: compare_class(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else value_replace(row[old_column]), axis=1)
    return df

df_1 = step_3(df_1)

In [316]:
def check_floor(string_1, string_2):
    """Установка маркера для изменения Этажа и Подвала/Цоколя"""
    list_words = set(['подвал', 'подвальный', 'цоколь', 'цокольный'])
    try:
        # проверка что поля имеют разный тип данных
        if isinstance(string_1, float) and isinstance(string_2, str) or isinstance(string_2, float) and isinstance(string_1, str):
            result = None
        # проверка что поля имеют одинаковый тип данных - nan(Not a Number) value
        elif isinstance(string_1, float) and isinstance(string_2, float):
            result = None
        # проверка что поля имеют одинаковый тип данных str
        elif type(string_1) == str and type(string_2) == str:
            # создание множества из строки
            values_1 = set(map(str.strip, string_1.split(';')))
            values_2 = set(map(str.strip, string_2.split(';')))
            # удаляем пустые элементы
            values_1.discard('')
            values_2.discard('')
            pattern = r'\s0+(\d+)'
            values_1 = {re.sub(pattern, r' \1', value) for value in values_1}
            values_2 = {re.sub(pattern, r' \1', value) for value in values_2}
            if (('Этаж 1' in values_1 and len(values_1) == 1) and not values_2.intersection({'Этаж 1'})) or (('Этаж 1' in values_2 and len(values_2) == 1) and not values_1.intersection({'Этаж 1'})):
                result = 'Появился или исчез 1 этаж'
            values_1 = ' '.join(map(str.lower, values_1))
            values_2 = ' '.join(map(str.lower, values_2))
            values_1 = set(values_1.split())
            values_2 = set(values_2.split())
            if values_1.intersection(list_words) and not values_2.intersection(list_words) or values_2.intersection(list_words) and not values_1.intersection(list_words):
                result = 'Появился или исчез подвал/цоколь'
            result = None
    except Exception as e:
        print(e)
    return result

In [317]:
def step_4(df):
    """Маркер для Этаж 1 и подвал/цоколь."""
    old_column = 'Сравнение: Местоположение в ОН (описание)'
    new_column = 'Маркер Этаж 1 и подвал/цоколь'
    previous_column = 'Пред.: Местоположение в ОН (описание)'
    present_column = 'Местоположение в ОН (описание)'
    
    df.insert(86, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: check_floor(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else None, axis=1)
    return df

df_1 = step_4(df_1)

In [318]:
def compare_location(string_1, string_2):
    """Сравнивает 2 поля - Пред.: Местоположение в ОН (описание) и Местоположение в ОН (описание).
    Делит строку по «;», убирает пробелы и создает множество, убирая 0 перед цифрой.
    """
    try:
        # проверка что поля имеют разный тип данных
        if isinstance(string_1, float) and isinstance(string_2, str) or isinstance(string_2, float) and isinstance(string_1, str):
            result = 'Различны'
        # проверка что поля имеют одинаковый тип данных - nan(Not a Number) value
        elif isinstance(string_1, float) and isinstance(string_2, float):
            result = 'Без изменений'
        # проверка что поля имеют одинаковый тип данных str
        elif type(string_1) == str and type(string_2) == str:
            # создание множества из строки
            values_1 = set(map(str.strip, string_1.split(';')))
            values_2 = set(map(str.strip, string_2.split(';')))
            # удаляем пустые элементы
            values_1.discard('')
            values_2.discard('')
            pattern = r'\s0+(\d+)'
            values_1 = {re.sub(pattern, r' \1', value) for value in values_1}
            values_2 = {re.sub(pattern, r' \1', value) for value in values_2}
            if values_1 == values_2:
                result = 'Без изменений'
            else:
                result = 'Различны'
    except Exception as e:
        print(e)
    return result

In [319]:
def step_5(df):
    """Сравнение: Местоположение в ОН (описание)."""
    old_column = 'Сравнение: Местоположение в ОН (описание)'
    new_column = 'Сравнение: Местоположение в ОН (описание) temp'
    previous_column = 'Пред.: Местоположение в ОН (описание)'
    present_column = 'Местоположение в ОН (описание)'
    
    df.insert(86, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: compare_location(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else value_replace(row[old_column]), axis=1)
    return df

df_1 = step_5(df_1)

In [320]:
# Путь к вспомогательным файлам
PATH_SUPPORT_DATA = 'L:/Беспалов Р.А/Обработка рынка Брянск/Вспомогательные файлы/'
PRIMORSKY_KRAI_SETTLEMENTS = 'Населенные_пункты_полигоны.xlsx'

def clean_settlements(file_name):
    """Извлекает населенные пункты и возвращает их в очищенном виде."""
    file_path = os.path.join(PATH_SUPPORT_DATA, file_name)
    df = pd.read_excel(file_path)

    df['NAME'] = df['NAME'].str.lower()
    df['NAME'] = df['NAME'].str.replace('ё', 'е')

    return set(df['NAME'])


settlements = clean_settlements(PRIMORSKY_KRAI_SETTLEMENTS)

In [321]:
def find_villages(data):
    """Находит Населенные пункты в строке.
    На вход принимает строку. В результате выводит множество НП, которые есть в строке.
    """

    # создаем шаблон для разделения строки
    city_list = []
    pattern = r'[ ,.\-:;!]+'
    
    # определяет только полное совпадение (работает когда поселок это одно слово)
    words = re.split(pattern, data)
    for word in words:
        word = {word}
        if word.intersection(settlements):
            city_list.append(word)

    # проверяет что хотя бы одно слово пересекается с названием населенного пункта, тогда проверяется ближайшее слово (+-1) вместе с найденным на полное соответствие населенному пункту
    for word in words:
        for village in settlements:
            lst = re.split(pattern, village)
            if word in lst:
                index = words.index(word)
                length = len(words) - 1
                if 0 <= index - 1 <= length:
                    if (words[index - 1] + ' ' + words[index] == village) or (words[index - 1] + '-' + words[index] == village):
                        city_list.append({village})

                if 0 <= index + 1 <= length:
                    if (words[index] + ' ' + words[index + 1] == village) or (words[index] + '-' + words[index + 1] == village):
                        city_list.append({village})

    if city_list:
        try:
            # city_set = set.union(*city_list)
            # return city_set
            return set(city_list[-1])
        except Exception as e:
            print(e)
    else:
        return None

In [322]:
def compare_address(string_1, string_2):
    """Сравнение адресов."""
    pattern_1 = r'["\']+'
    pattern_2 = r'\s{2,}'
    try:
        if isinstance(string_1, float) and isinstance(string_2, str) or isinstance(string_2, float) and isinstance(string_1, str):
            result = 'Различны'
        elif isinstance(string_1, float) and isinstance(string_2, float):
            result = 'Без изменений'
        elif type(string_1) == str and type(string_2) == str:
            string_1 = string_1.lower().replace('ё', 'е').strip()
            string_2 = string_2.lower().replace('ё', 'е').strip()

            string_1 = re.sub(pattern_1, '', string_1)
            string_1 = re.sub(pattern_2, ' ', string_1)

            string_2 = re.sub(pattern_1, '', string_2)
            string_2 = re.sub(pattern_2, ' ', string_2)

            set_1 = find_villages(string_1)
            set_2 = find_villages(string_2)

            if set_1 and set_2:
                if set_1 == set_2:
                    result = 'Без изменений'
                else:
                    result = 'Различны'
            elif set_1 is None and set_2 or set_2 is None and set_1:
                result = 'Различны'
            else: # None и # None
                result = 'Без изменений'
    except Exception as e:
        print(e)
    return result

In [323]:
def step_6(df):
    """Сравнение: Неформализованное описание адреса."""
    old_column = 'Сравнение: Неформализованное описание адреса'
    new_column = 'Сравнение: Неформализованное описание адреса temp'
    previous_column = 'Пред.: Неформализованное описание адреса'
    present_column = 'Неформализованное описание адреса'

    df.insert(83, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: compare_address(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else value_replace(row[old_column]), axis=1)
    return df

df_1 = step_6(df_1)

In [324]:
def step_7(df):
    """Сравнение: Наименование населённого пункта."""
    old_column = 'Сравнение: Наименование населённого пункта'
    new_column = 'Сравнение: Наименование населённого пункта temp'
    previous_column = 'Пред.: Наименование населённого пункта'
    present_column = 'Наименование населённого пункта'

    df.insert(74, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: compare_address(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else value_replace(row[old_column]), axis=1)
    return df

df_1 = step_7(df_1)

In [325]:
def region_compare(string_1, string_2):
    """Сравнивает наименование муниципального образования."""
    pattern_1 = r'["\']+'
    pattern_2 = r'\s{2,}'
    try:
        # проверка что поля имеют разный тип данных
        if isinstance(string_1, float) and isinstance(string_2, str) or isinstance(string_2, float) and isinstance(string_1, str):
            result = 'Различны'
        # проверка что поля имеют одинаковый тип данных - nan(Not a Number) value
        elif isinstance(string_1, float) and isinstance(string_2, float):
            result = 'Без изменений'
        # проверка что поля имеют одинаковый тип данных str
        elif type(string_1) == str and type(string_2) == str:
            string_1 = string_1.lower().replace('ё', 'е').strip()
            string_2 = string_2.lower().replace('ё', 'е').strip()

            string_1 = re.sub(pattern_1, '', string_1)
            string_1 = re.sub(pattern_2, ' ', string_1)

            string_2 = re.sub(pattern_1, '', string_2)
            string_2 = re.sub(pattern_2, ' ', string_2)

            if string_1 == string_2 or string_1.startswith(string_2) or string_2.startswith(string_2):
                result = 'Без изменений'
            else:
                result = 'Различны'
    except Exception as e:
        print(e)
    return result

In [326]:
def step_8(df):
    """Сравнение: Наименование муниципального образования (города)."""
    old_column = 'Сравнение: Наименование муниципального образования (города)'
    new_column = 'Сравнение: Наименование муниципального образования (города) temp'
    previous_column = 'Пред.: Наименование муниципального образования (города)'
    present_column = 'Наименование муниципального образования (города)'

    df.insert(71, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: region_compare(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else value_replace(row[old_column]), axis=1)
    return df

df_1 = step_8(df_1)

In [327]:
def simple_compare(string_1, string_2):
    """Сравнивает 2 поля простым сравнением."""
    pattern_1 = r'["\']+'
    pattern_2 = r'\s{2,}'
    pattern_3 = r'[?÷]+'
    try:
        # проверка что поля имеют разный тип данных
        if isinstance(string_1, float) and isinstance(string_2, str) or isinstance(string_2, float) and isinstance(string_1, str):
            result = 'Различны'
        # проверка что поля имеют одинаковый тип данных - nan(Not a Number) value
        elif isinstance(string_1, float) and isinstance(string_2, float):
            result = 'Без изменений'
        # проверка что поля имеют одинаковый тип данных str
        elif type(string_1) == str and type(string_2) == str:
            string_1 = string_1.lower().replace('ё', 'е').strip()
            string_2 = string_2.lower().replace('ё', 'е').strip()

            string_1 = re.sub(pattern_1, '', string_1)
            string_1 = re.sub(pattern_2, ' ', string_1)
            string_1 = re.sub(pattern_3, ' ', string_1)

            string_2 = re.sub(pattern_1, '', string_2)
            string_2 = re.sub(pattern_2, ' ', string_2)
            string_2 = re.sub(pattern_3, ' ', string_2)

            if string_1 == string_2:
                result = 'Без изменений'
            else:
                result = 'Различны'
    except Exception as e:
        print(e)
    return result

In [328]:
def step_9(df):
    """Сравнение: Наименование района."""
    old_column = 'Сравнение: Наименование района'
    new_column = 'Сравнение: Наименование района temp'
    previous_column = 'Пред.: Наименование района'
    present_column = 'Наименование района'

    df.insert(68, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: simple_compare(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else value_replace(row[old_column]), axis=1)
    return df

df_1 = step_9(df_1)

In [329]:
def compare_amount_floors(string_1, string_2):
    """Сравнивает 2 поля - количество этажей.
    Делит строку по «,», убирает пробелы и создает множество.
    """
    pattern = r',|\sи\s'
    try:
        # проверка что поля имеют разный тип данных
        if isinstance(string_1, float) and isinstance(string_2, str) or isinstance(string_2, float) and isinstance(string_1, str):
            result = 'Различны'
        # проверка что поля имеют одинаковый тип данных - nan(Not a Number) value
        elif isinstance(string_1, float) and isinstance(string_2, float):
            result = 'Без изменений'
        # проверка что поля имеют одинаковый тип данных str
        elif type(string_1) == str and type(string_2) == str:
            # создание множества из строки
            values_1 = set(map(str.strip, re.split(pattern, string_1)))
            values_2 = set(map(str.strip, re.split(pattern, string_2)))
            # удаляем пустые элементы
            values_1.discard('')
            values_2.discard('')
            if values_1 == values_2:
                result = 'Без изменений'
            else:
                result = 'Различны'
    except Exception as e:
        print(e)
    return result

In [330]:
def step_10(df):
    """Сравнение: Количество подземных этажей."""
    old_column = 'Сравнение: Количество подземных этажей'
    new_column = 'Сравнение: Количество подземных этажей temp'
    previous_column = 'Пред.: Количество подземных этажей'
    present_column = 'Количество подземных этажей'
    building_type = 'Вид объекта недвижимости'

    df.insert(62, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: compare_amount_floors(row[previous_column], row[present_column]) if row[old_column] == 'Различны' and row[building_type] == 'Здание' else value_replace(row[old_column]), axis=1)
    return df

df_1 = step_10(df_1)

In [331]:
def step_11(df):
    """Сравнение: Количество этажей (в том числе подземных)."""
    old_column = 'Сравнение: Количество этажей (в том числе подземных)'
    new_column = 'Сравнение: Количество этажей (в том числе подземных) temp'
    previous_column = 'Пред.: Количество этажей (в том числе подземных)'
    present_column = 'Количество этажей (в том числе подземных)'

    df.insert(59, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: compare_amount_floors(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else value_replace(row[old_column]), axis=1)
    return df

df_1 = step_11(df_1)

In [332]:
def marker_1(string_1, string_2):
    """Сравнивает года."""
    try:
        # проверка что поля имеют разный тип данных
        if isinstance(string_1, float) and isinstance(string_2, str) or isinstance(string_2, float) and isinstance(string_1, str):
            if isinstance(string_1, str):
                string_1 = string_1.strip()
                if len(string_1) != 4 or string_1.startswith('0'):
                    result = 'ошибка в годе'
            elif isinstance(string_2, str):
                string_2 = string_2.strip()
                if len(string_2) != 4 or string_2.startswith('0'):
                    result = 'ошибка в годе'
            else:
                result = None
        # проверка что поля имеют одинаковый тип данных - nan(Not a Number) value
        elif isinstance(string_1, float) and isinstance(string_2, float):
            result = None
        # проверка что поля имеют одинаковый тип данных str
        elif type(string_1) == str and type(string_2) == str:
            string_1 = string_1.strip()
            string_2 = string_2.strip()
            if len(string_1) != 4 or len(string_2) != 4 or string_1.startswith('0') or string_2.startswith('0'):
                result = 'ошибка в годе'
            else:
                result = None
    except Exception as e:
        print(e)
    return result

In [333]:
def step_12(df):
    """Маркер - год ввода в эксплуатацию."""
    old_column = 'Сравнение: Год ввода в эксплуатацию'
    new_column = 'Маркер - год ввода в эксплуатацию'
    previous_column = 'Пред.: Год ввода в эксплуатацию'
    present_column = 'Год ввода в эксплуатацию'
    
    df.insert(50, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: marker_1(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else None, axis=1)
    return df

df_1 = step_12(df_1)

In [334]:
def step_13(df):
    """Сравнение: Год ввода в эксплуатацию."""
    old_column = 'Сравнение: Год ввода в эксплуатацию'
    new_column = 'Сравнение: Год ввода в эксплуатацию temp'
    previous_column = 'Пред.: Год ввода в эксплуатацию'
    present_column = 'Год ввода в эксплуатацию'
    
    df.insert(50, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: simple_compare(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else value_replace(row[old_column]), axis=1)
    return df

df_1 = step_13(df_1)

In [335]:
def step_14(df):
    """Маркер - год завершения строительства."""
    old_column = 'Сравнение: Год завершения строительства'
    new_column = 'Маркер - год завершения строительства.'
    previous_column = 'Пред.: Год завершения строительства'
    present_column = 'Год завершения строительства'

    df.insert(47, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: marker_1(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else None, axis=1)
    return df

df_1 = step_14(df_1)

In [336]:
def step_15(df):
    """Сравнение: Год завершения строительства."""
    old_column = 'Сравнение: Год завершения строительства'
    new_column = 'Сравнение: Год завершения строительства temp'
    previous_column = 'Пред.: Год завершения строительства'
    present_column = 'Год завершения строительства'

    df.insert(47, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: simple_compare(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else value_replace(row[old_column]), axis=1)
    return df

df_1 = step_15(df_1)

In [337]:
def step_16(df):
    """Сравнение: Кадастровый номер здания или сооружения, в котором расположен ОН."""
    old_column = 'Сравнение: Кадастровый номер здания или сооружения, в котором расположен ОН'
    new_column = 'Сравнение: Кадастровый номер здания или сооружения, в котором расположен ОН temp'
    previous_column = 'Пред.: Кадастровый номер здания или сооружения, в котором расположен ОН'
    present_column = 'Кадастровый номер здания или сооружения, в котором расположен ОН'

    df.insert(41, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: simple_compare(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else value_replace(row[old_column]), axis=1)
    return df

df_1 = step_16(df_1)

In [338]:
def step_17(df):
    """Сравнение: Назначение сооружения."""
    old_column = 'Сравнение: Назначение сооружения'
    new_column = 'Сравнение: Назначение сооружения temp'
    previous_column = 'Пред.: Назначение сооружения'
    present_column = 'Назначение сооружения'

    df.insert(29, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: simple_compare(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else value_replace(row[old_column]), axis=1)
    return df

df_1 = step_17(df_1)

In [339]:
def step_18(df):
    """Сравнение: Наименование ОН."""
    old_column = 'Сравнение: Наименование ОН'
    new_column = 'Сравнение: Наименование ОН temp'
    previous_column = 'Пред.: Наименование ОН'
    present_column = 'Наименование ОН'

    df.insert(26, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: simple_compare(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else value_replace(row[old_column]), axis=1)
    return df

df_1 = step_18(df_1)

In [340]:
def step_19(df):
    """Сравнение: Кадастровый квартал."""
    old_column = 'Сравнение: Кадастровый квартал'
    new_column = 'Сравнение: Кадастровый квартал temp'
    previous_column = 'Пред.: Кадастровый квартал'
    present_column = 'Кадастровый квартал'

    df.insert(8, new_column, pd.Series)
    df[new_column] = df.apply(lambda row: simple_compare(row[previous_column], row[present_column]) if row[old_column] == 'Различны' else value_replace(row[old_column]), axis=1)
    return df

df_1 = step_19(df_1)

In [ ]:
def step_20(df):
    """Замена всех «Удалено» на «Без изменений»"""
    pass


## 3. Сохранение

In [341]:
excel_name = df_1.at[0, 'Имя файла']
df_1 = df_1.drop(columns='Имя файла')

In [342]:
df_1.to_excel(MODIFIED_DIR + excel_name, index=False)

In [343]:
end_time = time.time()


def execution_time(start, end):
    """Вычисляет время выполнения скрипта."""
    seconds = round(end - start, 1)
    minutes = round(seconds / 60, 1)
    return f"Программа выполнялась {seconds} секунд / {minutes} минут"

print(execution_time(start_time, end_time))

Программа выполнялась 92.2 секунд / 1.5 минут
